In [ ]:
from PIL import Image
import os

def resize_images_and_labels(image_dir, label_dir, output_size=(640, 640)):
    for filename in os.listdir(image_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Charger l'image
            image_path = os.path.join(image_dir, filename)
            image = Image.open(image_path)
            original_size = image.size
            
            # Redimensionner l'image
            resized_image = image.resize(output_size, Image.Resampling.LANCZOS)
            resized_image.save(image_path)  # Enregistrer l'image redimensionnée
            
            # Adapter les labels
            label_path = os.path.join(label_dir, filename.replace('jpg', 'txt').replace('png', 'txt'))
            if os.path.exists(label_path):
                with open(label_path, 'r') as file:
                    lines = file.readlines()
                
                with open(label_path, 'w') as file:
                    for line in lines:
                        parts = line.split()
                        if len(parts) == 5:
                            x_center = float(parts[1]) * original_size[0]
                            y_center = float(parts[2]) * original_size[1]
                            width = float(parts[3]) * original_size[0]
                            height = float(parts[4]) * original_size[1]
                            
                            # Calculer les nouvelles coordonnées
                            x_center = (x_center / original_size[0]) * output_size[0]
                            y_center = (y_center / original_size[1]) * output_size[1]
                            width = (width / original_size[0]) * output_size[0]
                            height = (height / original_size[1]) * output_size[1]
                            
                            # Enregistrer les nouvelles coordonnées
                            file.write(f"{parts[0]} {x_center/output_size[0]:.6f} {y_center/output_size[1]:.6f} {width/output_size[0]:.6f} {height/output_size[1]:.6f}\n")

resize_images_and_labels('data/images2', 'data/labels2')

In [ ]:
import cv2
import matplotlib.pyplot as plt

def show_image_with_boxes(image_path, label_path):
    # Charger l'image
    image = cv2.imread(image_path)
    
    # Lire les labels à partir du fichier
    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    # Dessiner les boîtes englobantes sur l'image
    for line in lines:
        parts = line.split()
        if len(parts) == 5:
            label = parts[0]
            x_center = float(parts[1]) * image.shape[1]
            y_center = float(parts[2]) * image.shape[0]
            width = float(parts[3]) * image.shape[1]
            height = float(parts[4]) * image.shape[0]
            
            x_min = int(x_center - width/2)
            y_min = int(y_center - height/2)
            x_max = int(x_center + width/2)
            y_max = int(y_center + height/2)
            
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Afficher l'image avec les boîtes englobantes
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
show_image_with_boxes('data/images2/P4022021.jpg', 'data/labels2/P4022021.txt')